# Merging Datasets in Pandas

## Combining Data from Multiple Sources

**Topics:**
- `pd.concat()` - Stacking DataFrames
- `pd.merge()` - Joining on keys
- `df.join()` - Index-based joins

## Setup

Import libraries and load our datasets

In [ ]:
import pandas as pd
%matplotlib inline
from IPython.display import display

## Example Datasets

We'll work with:
- WDI economic data
- Country area data
- Population data

In [ ]:
# WDI data - GDP components
url = "https://datascience.quantecon.org/assets/data/wdi_data.csv"
wdi = pd.read_csv(url).set_index(["country", "year"])
wdi2017 = wdi.xs(2017, level="year")
wdi2017.head()

In [ ]:
# Country area in square miles
sq_miles = pd.Series({
   "United States": 3.8,
   "Canada": 3.8,
   "Germany": 0.137,
   "United Kingdom": 0.0936,
   "Russia": 6.6,
}, name="sq_miles").to_frame()
sq_miles.index.name = "country"
sq_miles

In [ ]:
# Population data
pop_url = "https://datascience.quantecon.org/assets/data/wdi_population.csv"
pop = pd.read_csv(pop_url).set_index(["country", "year"])
pop.head()

## pd.concat(): Stacking DataFrames

Use `pd.concat()` to stack DataFrames:
- `axis=0`: Stack vertically (rows)
- `axis=1`: Stack horizontally (columns)

### axis=0: Vertical Stacking

In [ ]:
pd.concat([wdi2017, sq_miles], axis=0)

### axis=1: Horizontal Stacking

In [ ]:
pd.concat([wdi2017, sq_miles], axis=1)

### Quick Analysis: Consumption per Square Mile

In [ ]:
temp = pd.concat([wdi2017, sq_miles], axis=1)
temp["Consumption"] / temp["sq_miles"]

## pd.merge(): Join on Keys

The most powerful combining tool:
- Aligns data based on key columns
- Similar to SQL joins
- Multiple merge strategies available

### Basic Merge Example

In [ ]:
pd.merge(wdi2017, sq_miles, on="country")

### Merging with Multiple Years

In [ ]:
wdi2016_17 = wdi.loc[pd.IndexSlice[:, [2016, 2017]],:]
pd.merge(wdi2016_17.reset_index(), sq_miles, on="country")

### Merging on Multiple Columns

In [ ]:
pd.merge(wdi2016_17, pop, on=["country", "year"])

### Computing Per Capita Metrics

In [ ]:
wdi_pop = pd.merge(wdi2016_17, pop, on=["country", "year"])

print("GDP per capita:")
display(wdi_pop["GDP"] / wdi_pop["Population"])

print("\nConsumption per capita:")
display(wdi_pop["Consumption"] / wdi_pop["Population"])

## Merge Arguments: The 'how' Parameter

Controls which keys appear in output:
- `left`: All keys from left DataFrame
- `right`: All keys from right DataFrame
- `inner`: Only keys in both (default)
- `outer`: All keys from either

In [ ]:
# Create test DataFrames
wdi2017_no_US = wdi2017.drop("United States")
sq_miles_no_germany = sq_miles.drop("Germany")

In [ ]:
# Left merge - keeps all left keys
pd.merge(wdi2017, sq_miles, on="country", how="left")

In [ ]:
# Right merge - keeps all right keys (note Russia included)
pd.merge(wdi2017, sq_miles, on="country", how="right")

In [ ]:
# Inner merge - only matching keys
pd.merge(wdi2017_no_US, sq_miles, on="country", how="inner")

In [ ]:
# Outer merge - all keys from either
pd.merge(wdi2017_no_US, sq_miles_no_germany, on="country", how="outer")

## df.join(): Simplified Merging

Convenience method for index-based joins:
- Automatically uses right DataFrame's index
- Equivalent to `merge()` with `right_index=True`

In [ ]:
wdi2017.join(sq_miles, on="country")

## Case Study: Goodreads Book Ratings

Analyzing 6 million ratings for 10,000 books

In [ ]:
url = "https://datascience.quantecon.org/assets/data/goodreads_ratings.csv.zip"
ratings = pd.read_csv(url)
display(ratings.head())
ratings.info()

### Rating Distribution

In [ ]:
ratings["rating"].value_counts().sort_index().plot(kind="bar", 
                                                     title="Distribution of Ratings");

### Loading Book Metadata

In [ ]:
url = "https://datascience.quantecon.org/assets/data/goodreads_books.csv"
books = pd.read_csv(url)
books = books[["book_id", "authors", "title"]]
books.head()

### Merging Ratings with Book Info

In [ ]:
rated_books = pd.merge(ratings, books)
rated_books.head()

### Most Rated Books

In [ ]:
most_rated_books_id = rated_books["book_id"].value_counts().nlargest(10).index
most_rated_books = rated_books.loc[rated_books["book_id"].isin(most_rated_books_id), :]
list(most_rated_books["title"].unique())

### Average Ratings for Popular Books

In [ ]:
most_rated_books.pivot_table(values="rating", index="title")

### Overall Rating Distribution

In [ ]:
average_ratings = (
    rated_books
    .pivot_table(values="rating", index="title")
    .sort_values(by="rating", ascending=False)
)

average_ratings.plot.hist(bins=30, xlim=(1, 5), 
                          title="Distribution of Average Book Ratings")

## Case Study: Airline Delays

US domestic flight delays from November 2016

In [ ]:
url = "https://datascience.quantecon.org/assets/data/airline_performance_dec16.csv.zip"
air_perf = pd.read_csv(url)[["CRSDepTime", "Carrier", "CarrierDelay", "ArrDelay"]]
air_perf.head()

### Average Carrier Delays

In [ ]:
avg_delays = (
    air_perf
    .pivot_table(index="Carrier", values="CarrierDelay", aggfunc="mean")
    .sort_values("CarrierDelay")
    .nlargest(10, "CarrierDelay")
)
avg_delays

### Adding Airline Names

In [ ]:
url = "https://datascience.quantecon.org/assets/data/airline_carrier_codes.csv.zip"
carrier_code = pd.read_csv(url)

avg_delays_w_code = pd.merge(avg_delays, carrier_code, 
                             left_on="Carrier", right_on="Code")
avg_delays_w_code.sort_values("CarrierDelay", ascending=False)

## Key Takeaways

**Three main combining methods:**
1. `pd.concat()` - Stack DataFrames vertically or horizontally
2. `pd.merge()` - Join on key columns with various strategies
3. `df.join()` - Simplified index-based joining

**Choose based on your needs:**
- Simple stacking → `concat()`
- Key-based alignment → `merge()`
- Index alignment → `join()`

## Practice Exercises

1. Compute population density for each country
2. Compare different merge strategies (`how` parameter)
3. Analyze least-rated books on Goodreads
4. Explore airline delay patterns further